In [ ]:
from manim import *
import numpy as np

class NepaliSolidCalculation(ThreeDScene):
    def construct(self):
        # ==========================================
        # 1. SETUP & CONFIGURATION
        # ==========================================
        
        # --- Dimensions ---
        L_val, B_val, H_box = 8, 6, 3.5
        R_val, H_cyl = 2.5, 4.5
        
        # --- Scaling ---
        s = 0.35 
        
        # --- Colors ---
        C_BOX = BLUE_D
        C_BOX_STR = BLUE_A
        C_CYL = TEAL_D
        C_CYL_STR = TEAL_A
        C_HEMI = RED_D
        C_HEMI_STR = RED_A
        C_DIM = YELLOW 
        
        font_np = "Noto Sans Devanagari" 

        # --- Positioning ---
        # 3D Object Center
        SOLID_CENTER = RIGHT * 3.8 + DOWN * 2.0
        
        # Z-axis Stacking
        z_box_center = (H_box * s) / 2
        z_cyl_base = H_box * s
        z_cyl_center = z_cyl_base + (H_cyl * s) / 2
        z_hemi_base = z_cyl_base + (H_cyl * s)

        # ==========================================
        # 2. HELPER FUNCTION (Fixes Distorted Lines)
        # ==========================================
        def get_dim_line(start, end, text, direction=UP, rotate_axis=None, rotate_angle=0):
            """
            Creates a thin line with tips (instead of a thick arrow) 
            to prevent 3D distortion.
            """
            # Create the line
            line = Line(start, end, color=C_DIM, stroke_width=2)
            line.add_tip(tip_length=0.15, tip_width=0.15)
            line.add_tip(tip_length=0.15, tip_width=0.15, at_start=True)
            
            # Create the label
            label = MathTex(text, color=C_DIM, font_size=28)
            
            # Position label relative to the line (default is usually 'UP' relative to line)
            # We use shift to move it slightly away from the line
            midpoint = (start + end) / 2
            label.move_to(midpoint)
            label.shift(direction * 0.3) 
            
            # Rotate text to align with the camera view if needed
            if rotate_axis is not None:
                label.rotate(rotate_angle, axis=rotate_axis)
                
            return VGroup(line, label), label

        # ==========================================
        # 3. OBJECT GENERATION
        # ==========================================
        
        # A. Box
        box = Cube(side_length=1, fill_opacity=0.8, fill_color=C_BOX)
        box.scale([L_val*s, B_val*s, H_box*s]) 
        box.set_stroke(width=2, color=C_BOX_STR)
        box.move_to(SOLID_CENTER + OUT * z_box_center)
        
        # B. Cylinder
        cylinder = Cylinder(
            radius=R_val*s, height=H_cyl*s, direction=OUT, 
            fill_opacity=0.8, fill_color=C_CYL
        )
        cylinder.set_stroke(width=0)
        cylinder.move_to(SOLID_CENTER + OUT * z_cyl_center)
        
        # Rings
        cyl_ring_bot = Circle(radius=R_val*s, color=C_CYL_STR).move_to(SOLID_CENTER + OUT*z_cyl_base)
        cyl_ring_top = Circle(radius=R_val*s, color=C_CYL_STR).move_to(SOLID_CENTER + OUT*z_hemi_base)

        # C. Hemisphere
        hemisphere = Surface(
            lambda u, v: np.array([
                (R_val*s) * np.cos(u) * np.sin(v),
                (R_val*s) * np.sin(u) * np.sin(v),
                (R_val*s) * np.cos(v)
            ]),
            u_range=[0, TAU],
            v_range=[0, PI/2],
            resolution=(24, 24)
        )
        hemisphere.set_fill(C_HEMI, opacity=0.8)
        hemisphere.set_style(stroke_width=0.5, stroke_color=C_HEMI_STR)
        hemisphere.shift(SOLID_CENTER + OUT * z_hemi_base)

        # ==========================================
        # 4. UI & TEXT SETUP (ADJUSTED SPACING)
        # ==========================================
        
        # Title - Moved UP higher
        title = Text("Combined Solid Volume", font_size=30).to_corner(UL).shift(UP*0.2)
        title_np = Text("संयुक्त ठोस वस्तुको आयतन", font=font_np, font_size=22, color=YELLOW)
        title_np.next_to(title, DOWN, buff=0.1, aligned_edge=LEFT)
        
        self.add_fixed_in_frame_mobjects(title, title_np)
        
        # Anchor - Shifted UP to make room
        calc_anchor = title_np.get_bottom() + DOWN * 0.3 + LEFT * 0.2

        # ==========================================
        # 5. ANIMATION
        # ==========================================
        
        self.set_camera_orientation(phi=60 * DEGREES, theta=-45 * DEGREES, zoom=0.7)
        
        # Build
        self.play(DrawBorderThenFill(box), run_time=1)
        self.play(
            GrowFromPoint(cylinder, SOLID_CENTER + OUT*z_cyl_base), 
            Create(cyl_ring_bot), Create(cyl_ring_top),
            run_time=1
        )
        self.play(Write(hemisphere), run_time=1)
        self.wait(0.5)

        # ------------------------------------------
        # STEP 1: BOX
        # ------------------------------------------
        txt_box = Text("1. Box (बाकस)", font=font_np, color=C_BOX, font_size=22)
        txt_box.move_to(calc_anchor, aligned_edge=LEFT)
        self.add_fixed_in_frame_mobjects(txt_box)
        self.play(Write(txt_box))

        # Camera: Front Low (Length)
        self.move_camera(phi=80*DEGREES, theta=-90*DEGREES, frame_center=box.get_center() + DOWN*1.5, zoom=1.3)
        
        # Length Line
        p1 = box.get_corner(DL+IN) 
        p2 = box.get_corner(DR+IN)
        l_grp, l_lbl = get_dim_line(p1, p2, f"l={L_val}", direction=DOWN, rotate_axis=RIGHT, rotate_angle=90*DEGREES)
        self.play(Create(l_grp))
        
        # Camera: Side (Breadth)
        self.move_camera(phi=70*DEGREES, theta=0*DEGREES, frame_center=box.get_center() + RIGHT*2, zoom=1.3)
        
        # Breadth Line
        p_start_b = box.get_center() + RIGHT*(L_val*s/2) + DOWN*(B_val*s/2) + OUT*(-H_box*s/2)
        p_end_b   = box.get_center() + RIGHT*(L_val*s/2) + UP*(B_val*s/2)   + OUT*(-H_box*s/2)
        b_grp, b_lbl = get_dim_line(p_start_b, p_end_b, f"b={B_val}", direction=RIGHT)
        # Fix rotation for side view
        b_lbl.rotate(90*DEGREES, RIGHT).rotate(90*DEGREES, OUT)
        self.play(Create(b_grp))

        # Camera: Height (Tilt)
        self.move_camera(phi=60*DEGREES, theta=20*DEGREES, frame_center=box.get_center(), zoom=1.3)
        
        # Height Line
        p_base_h = box.get_center() + RIGHT*(L_val*s/2) + UP*(B_val*s/2) + OUT*(-H_box*s/2)
        p_top_h  = box.get_center() + RIGHT*(L_val*s/2) + UP*(B_val*s/2) + OUT*(H_box*s/2)
        h_grp, h_lbl = get_dim_line(p_base_h, p_top_h, f"h={H_box}", direction=RIGHT)
        h_lbl.rotate(90*DEGREES, RIGHT).rotate(90*DEGREES, OUT)
        self.play(Create(h_grp))

        # Calc Box (Smaller text, tighter spacing)
        eq_box = MathTex(r"V_{box} = l \times b \times h", font_size=26)
        eq_box_nums = MathTex(f"= {L_val} \\times {B_val} \\times {H_box}", font_size=26)
        eq_box_res = MathTex(f"= {round(L_val*B_val*H_box, 2)}", font_size=26, color=C_BOX)
        
        grp_box_math = VGroup(eq_box, eq_box_nums, eq_box_res).arrange(DOWN, buff=0.1, aligned_edge=LEFT)
        grp_box_math.next_to(txt_box, DOWN, buff=0.1, aligned_edge=LEFT)
        self.add_fixed_in_frame_mobjects(grp_box_math)
        self.play(Write(grp_box_math))

        # ------------------------------------------
        # STEP 2: CYLINDER
        # ------------------------------------------
        txt_cyl = Text("2. Cylinder (बेलना)", font=font_np, color=C_CYL, font_size=22)
        # Reduced buff here
        txt_cyl.next_to(grp_box_math, DOWN, buff=0.3, aligned_edge=LEFT)
        self.add_fixed_in_frame_mobjects(txt_cyl)
        self.play(FadeIn(txt_cyl))

        # Camera: Side
        self.move_camera(phi=75*DEGREES, theta=10*DEGREES, frame_center=cylinder.get_center(), zoom=1.5, run_time=1.5)

        # Height Line
        p_cyl_bot = cylinder.get_center() + RIGHT*(R_val*s) + OUT*(-H_cyl*s/2)
        p_cyl_top = cylinder.get_center() + RIGHT*(R_val*s) + OUT*(H_cyl*s/2)
        hc_grp, hc_lbl = get_dim_line(p_cyl_bot, p_cyl_top, f"h={H_cyl}", direction=RIGHT)
        hc_lbl.rotate(90*DEGREES, RIGHT).rotate(90*DEGREES, OUT)
        self.play(Create(hc_grp))

        # Calc Cylinder
        eq_cyl = MathTex(r"V_{cyl} = \pi r^2 h", font_size=26)
        eq_cyl_nums = MathTex(f"= \pi ({R_val})^2 ({H_cyl})", font_size=26)
        vol_c = np.pi * (R_val**2) * H_cyl
        eq_cyl_res = MathTex(f"\\approx {round(vol_c, 2)}", font_size=26, color=C_CYL)
        
        grp_cyl_math = VGroup(eq_cyl, eq_cyl_nums, eq_cyl_res).arrange(DOWN, buff=0.1, aligned_edge=LEFT)
        grp_cyl_math.next_to(txt_cyl, DOWN, buff=0.1, aligned_edge=LEFT)
        self.add_fixed_in_frame_mobjects(grp_cyl_math)
        self.play(Write(grp_cyl_math))

        # ------------------------------------------
        # STEP 3: HEMISPHERE
        # ------------------------------------------
        txt_hemi = Text("3. Hemisphere (अर्धगोला)", font=font_np, color=C_HEMI, font_size=22)
        txt_hemi.next_to(grp_cyl_math, DOWN, buff=0.3, aligned_edge=LEFT)
        self.add_fixed_in_frame_mobjects(txt_hemi)
        self.play(FadeIn(txt_hemi))

        # Camera: Top
        self.move_camera(phi=0*DEGREES, theta=-90*DEGREES, frame_center=hemisphere.get_center(), zoom=1.8, run_time=1.5)

        # Radius Line
        p_center = SOLID_CENTER + OUT*z_hemi_base
        p_edge   = p_center + RIGHT*(R_val*s)
        # Note: In top view (phi=0), text is naturally legible if not rotated weirdly
        r_grp, r_lbl = get_dim_line(p_center, p_edge, f"r={R_val}", direction=UP)
        self.play(Create(r_grp))

        # Calc Hemisphere
        eq_hemi = MathTex(r"V_{hemi} = \frac{2}{3} \pi r^3", font_size=26)
        eq_hemi_nums = MathTex(f"= \\frac{{2}}{{3}} \pi ({R_val})^3", font_size=26)
        vol_h = (2/3) * np.pi * (R_val**3)
        eq_hemi_res = MathTex(f"\\approx {round(vol_h, 2)}", font_size=26, color=C_HEMI)

        grp_hemi_math = VGroup(eq_hemi, eq_hemi_nums, eq_hemi_res).arrange(DOWN, buff=0.1, aligned_edge=LEFT)
        grp_hemi_math.next_to(txt_hemi, DOWN, buff=0.1, aligned_edge=LEFT)
        self.add_fixed_in_frame_mobjects(grp_hemi_math)
        self.play(Write(grp_hemi_math))

        # ------------------------------------------
        # STEP 4: TOTAL
        # ------------------------------------------
        
        self.move_camera(phi=65*DEGREES, theta=-45*DEGREES, frame_center=SOLID_CENTER + OUT*2, zoom=0.55, run_time=2.5)
        
        self.play(
            FadeOut(l_grp), FadeOut(b_grp), FadeOut(h_grp),
            FadeOut(hc_grp), FadeOut(r_grp)
        )

        divider = Line(LEFT, RIGHT, color=WHITE).set_length(3.5)
        divider.next_to(grp_hemi_math, DOWN, buff=0.2).align_to(grp_hemi_math, LEFT)
        
        total_val = (L_val*B_val*H_box) + vol_c + vol_h
        
        txt_total_lbl = Text("Total Volume (जम्मा आयतन)", font=font_np, font_size=22, color=YELLOW)
        txt_total_math = MathTex(f"V \\approx {round(total_val, 2)}", font_size=30, color=YELLOW)
        
        grp_total = VGroup(divider, txt_total_lbl, txt_total_math).arrange(DOWN, buff=0.1, aligned_edge=LEFT)
        grp_total.next_to(grp_hemi_math, DOWN, buff=0.2, aligned_edge=LEFT)
        
        self.add_fixed_in_frame_mobjects(grp_total)
        self.play(Write(grp_total))
        
        self.begin_ambient_camera_rotation(rate=0.2)
        self.wait(3)

%manim -ql -v warning NepaliSolidCalculation

Manim Community v0.19.1